In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import time
from ipywidgets import interact, fixed


Wygenerowac tablice N punktow (x,y)

In [ ]:
np.random.seed(46)
def secret_fun(x):
    return x * np.sin(x/2) + np.log(x) * np.cos(x**3) + 30 *np.random.rand()
#     return x * np.random.rand() + np.random.rand()


In [ ]:
X = np.linspace(1, 100, 30)
Y = np.array([secret_fun(x) for x in X])
more_X = np.linspace(1, 100, 300)
more_Y = np.array([secret_fun(x) for x in more_X])

plt.scatter(X, Y)
plt.plot(more_X, more_Y, color='red')
plt.show()

In [ ]:
src_df = pd.DataFrame({'x': X, 'y':Y})
src_df.to_csv('secret_fun.csv', index=False)

Uzyc funkcji gsl do interpolacji wielomianowej dla tych punktow - uzyc gsl_interp_polynomial. Narysowac jego wykres.

In [ ]:
gsl_interp_df = pd.read_csv('gsl_interp.csv')
list(gsl_interp_df)

In [ ]:
plt.scatter(X, Y)
plt.plot(gsl_interp_df['x'], gsl_interp_df['y'], color='red')
plt.plot(more_X, more_Y, color='green')
plt.show()

Napisac wlasny program generujacy dane (recznie - bez korzystania z gsl) do narysowania wykresu wielomianu interpolujacego metoda Lagrange'a dla tych punktow w wybranym przedziale. Postarac sie zaprojektowac API podobnie do GSL - osobna funkcja init oraz eval Narysowac wykres.

In [ ]:
def interpolation_nodes(XY, nodes_num):
    all_points_no = XY.shape[0]
    assert(nodes_num <= all_points_no)
    nodes = np.array([XY[round(i*(all_points_no / nodes_num))] for i in range(nodes_num)])
    nodes[nodes_num - 1] = XY[all_points_no - 1]
    return nodes

In [ ]:
# TODO vectorization
def lagrange_poly(X, Y, nodes_num=None):
    nodes_num = len(X) if nodes_num is None else nodes_num
    XY = np.c_[X,Y]
    nodes = interpolation_nodes(XY, nodes_num)
    
    def basis(m, j):
        if m == j:
            return lambda x: 1
        return lambda x: (x - nodes[m][0] ) / (nodes[j][0] - nodes[m][0])
    
    def result(x):
        s = 0
        for j in range(len(nodes)):
            l = nodes[j][1]
            for m in range(len(nodes)):
                l *= basis(m, j)(x)
            s += l
        return s
    return result, nodes                    
        

In [ ]:
def demonstrate_lagrange(max_X, interp_nodes_no):
    X = np.linspace(1, max_X, 30)
    more_X = np.linspace(1, max_X, 300)
    Y = np.array([secret_fun(x) for x in X])
    poly, nodes = lagrange_poly(X, Y, nodes_num=interp_nodes_no)
    more_Y_interp = poly(more_X)
    more_Y = np.array([secret_fun(x) for x in more_X])
    plt.scatter(X, Y, color='blue')
    plt.scatter([n[0] for n in nodes], [n[1] for n in nodes], color='red')
    plt.plot(more_X, more_Y_interp, color='red')
    plt.plot(more_X, more_Y, color='green')

    plt.show()

In [ ]:
interact(demonstrate_lagrange, 
         max_X=widgets.IntSlider(min=1, max=1000, value=100),
        interp_nodes_no=widgets.IntSlider(min=1, max=30, value=30)
        )

Zrobic to samo metoda Newtona. Porownac wszystkie 3 wyniki na jednym wykresie.

In [ ]:
def newton_poly(X, Y):
    XY = np.c_[X,Y]
    cache = [[None for j in range(len(XY))] for i in range(len(XY))]
    
    def div_difference(i, j):
        assert(i <= j)
        if(i == j):
            return XY[i][1]
        if cache[i][j] is None:
            cache[i][j] =  (div_difference(i + 1, j) - div_difference(i, j-1)) / (XY[j][0] - XY[i][0]) 
        return cache[i][j]
    
    def mul_up_to(i):
        def result(x):
            r = 1
            for j in range(i):
#                 print(i)
                r *= (x - XY[j][0])
            return r
        return result
    
    A = np.array([div_difference(0,j) for j in range(len(XY))])
    def result(x):
        muls = np.array([mul_up_to(i)(x) for i in range(len(XY))])
        return (A * muls).sum()
    
    return result
    

In [ ]:
def demonstrate_newton(max_X):
    X = np.linspace(1, max_X, 30)
    more_X = np.linspace(1, max_X, 300)
    Y = np.array([secret_fun(x) for x in X])
    poly = newton_poly(X, Y)
    more_Y_interp = [poly(x) for x in more_X]
    plt.scatter(X, Y, color='blue')
    plt.plot(more_X, more_Y_interp, color='red')
    plt.show()

In [ ]:
interact(demonstrate_newton, 
         max_X=widgets.IntSlider(min=1, max=1000, value=100, step=10)
        )

In [ ]:
def demonstrate_all(max_X,lagrange_interp_nodes_no):
    X = np.linspace(1, max_X, 30)
    more_X = np.linspace(1, max_X, 300)
    Y = np.array([secret_fun(x) for x in X])
    
    lagrange_p, nodes = lagrange_poly(X, Y, nodes_num=lagrange_interp_nodes_no)
    lagrange_Y_interp = np.array([lagrange_p(x) for x in more_X])
    
    newton_p = newton_poly(X, Y)
    newton_Y_interp = np.array([newton_p(x) for x in more_X])
    
    plt.scatter(X, Y, color='blue')
    plt.scatter([n[0] for n in nodes], [n[1] for n in nodes], color='red')

    plt.plot(gsl_interp_df['x'], gsl_interp_df['y'], color='green')
    
    plt.plot(more_X, newton_Y_interp, color='orange')
    plt.plot(more_X, lagrange_Y_interp, color='red')
    print((newton_Y_interp - lagrange_Y_interp).sum())
    plt.show()

In [ ]:
interact(demonstrate_all, 
         max_X=widgets.IntSlider(min=1, max=1000, value=100),
        lagrange_interp_nodes_no=widgets.IntSlider(min=1, max=30, value=30)
        )


Porownac metody poprzez pomiar czasu wykonania dla zmiennej ilosci wezlow interpolacji. Dokonac pomiaru 10 razy i policzyc wartosc srednia oraz oszacowac blad pomiaru za pomoca odchylenia standardowego. Narysowac wykresy w R.

In [ ]:
def interp_create_time(poly_generator, nodes_count, max_X=1000):
    X = np.linspace(1, max_X, nodes_count)
    Y = np.array([secret_fun(x) for x in X])
    t_0 = time.time()
    poly = poly_generator(X, Y)
    t_1 = time.time()
    return poly, t_1 - t_0

In [ ]:
def interp_perform_time(poly, max_X=1000):
    more_X = np.linspace(1, max_X, max_X)
    t_0 = time.time()
    interp = np.array([poly(x) for x in more_X])
    t_1 = time.time()
    return t_1 - t_0

In [ ]:
def performance(generator, max_nodes, is_lagrange=False):
    creation_times = []
    performance_times = []
    for nodes_count in range(1, max_nodes):
        poly, creation_time = interp_create_time(generator, nodes_count)
        if is_lagrange:
            poly = poly[0]
        interp_time = interp_perform_time(poly)
        creation_times.append(creation_time)
        performance_times.append(interp_time)
    return creation_times, performance_times    

In [ ]:
max_nodes = 30
l_creation, l_performance = performance(lagrange_poly, max_nodes, True)
n_creation, n_performance = performance(newton_poly, max_nodes)
nodes_list = list(range(1, max_nodes))
plt.plot(nodes_list, l_creation, color='red')
plt.plot(nodes_list, n_creation, color='blue')
plt.title('interpolation generation time')
plt.show()

In [ ]:
plt.plot(nodes_list, l_performance, color='red')
plt.plot(nodes_list, n_performance, color='blue')
plt.title('interpolation evaluation time')
plt.show()

Poeksperymentowac z innymi typami interpolacji gsl (cspline, akima), zmierzyc czasy, narysowac wykresy i porownac z wykresami interpolacji wielomianowej. Zaobserwowac, gdzie wystepuje efekt Rungego.

In [ ]:
from scipy import interpolate

def demonstrate_cspline(max_X, interp_nodes):
    X = np.linspace(1, max_X, interp_nodes)
    Y = secret_fun(X)
    more_X = np.linspace(1, max_X, max_X)
    tck = interpolate.splrep(X, Y, s=0)
    more_Y_interp = interpolate.splev(more_X, tck, der=0)
    more_Y = secret_fun(more_X)
    plt.scatter(X, Y, color='blue')
    plt.plot(more_X, more_Y_interp, color='red')
    plt.plot(more_X, more_Y, color='green')

    plt.show()


In [ ]:
interact(demonstrate_cspline, 
         max_X=widgets.IntSlider(min=1, max=1000, value=100, step=10),
         interp_nodes=widgets.IntSlider(min=1, max=100, value=30)
        )

In [ ]:
def demonstrate_akima(max_X, interp_nodes):
    X = np.linspace(1, max_X, interp_nodes)
    Y = secret_fun(X)
    more_X = np.linspace(1, max_X, max_X)
    interpolator = interpolate.Akima1DInterpolator(X, Y)
    more_Y_interp = interpolator(more_X)
    more_Y = secret_fun(more_X)
    plt.scatter(X, Y, color='blue')
    plt.plot(more_X, more_Y_interp, color='red')
    plt.plot(more_X, more_Y, color='green')

    plt.show()


In [ ]:
interact(demonstrate_cspline, 
         max_X=widgets.IntSlider(min=1, max=1000, value=100, step=10),
         interp_nodes=widgets.IntSlider(min=1, max=100, value=30)
        )